In [68]:
#Import libraries
import sympy as sp
import numpy as np
np.set_printoptions(legacy='1.25')

## Problem 1

Consider the matrix

$ A = \begin{bmatrix}
3 & 1 & 6 \\
3 & 0 & 4 \\
1 & 3 & 2 
\end{bmatrix}  $

in GF(7).

(a) Find the determinant of $A$. Is it invertible?

>A matrix is invertible if it is a square $n$ by $n$ matrix, and the determinant is non-zero.

In [69]:
def find_modulo_inverse(input, modulo) : #Finds inverse in modulo.
    #e.g. 2 inverse in mod 3 is 2 b/c 2*2 = 4(mod 3) = 1
    #==Inputs== 
    #input: value to find the inverse of
    #modulo: value of the modulus
    i=modulo
    while i>0 :
        if np.mod(input*i,modulo) == 1 :
            return i
        i-=1
    return 0

def recursive_det(matrix, p): # Recursively finds the determinant of a matrix in GF(p)
    #base case: 2x2 matrix
    #output: determinant of a matrix.

    if len(matrix) == 2 : #base case
        return round(np.linalg.det(matrix))
    else :
        i=0
        j=0
        while j<len(matrix) :
            coefficient = (-1)**(i+j)
            submatrix = np.delete((np.delete(matrix, i,axis=0)),j,axis=1)
            value = recursive_det(submatrix,p) #recursion
            value2 = coefficient*value*matrix[i][j]
            if i==0 :
                if j==0 : accum = value2 % p
                else : accum = (accum + value2) % p
            j+=1

    return accum

In [70]:
A = np.array([[3,1,6],[3,0,4],[1,3,2]])

print(f'The determinant of A = {recursive_det(A,7)}')

The determinant of A = 2


> The determinant of A = 2, which is not zero, therefore A is invertible.

> I used numpy's numpy.linalg.det() function which returns the determinant of a matrix. I take the modulus of the result.

(b) Recall that in real numbers, the inverse of $A$ can be found as

$ A^{-1} = \frac{1}{det(A)} \begin{bmatrix}
det(A(1, 1)) & − det(A(1, 2)) & det(A(1, 3)) \\
− det(A(2, 1)) & det(A(2, 2)) & − det(A(2, 3)) \\
det(A(3, 1)) & − det(A(3, 2)) & det(A(3, 3)) 
\end{bmatrix}  $

where $A(i, j)$ is the matrix obtained by removing row $i$ and column $j$ of matrix $A$. For instance,

$ A(1,2) = \begin{bmatrix}
3 & 4 \\
1 & 2 
\end{bmatrix}  $

Note that $±$ det($A(i, j)$) is still an integer, and hence, you can find the inverse of $A$ in GF(7) by replacing division by det($A$) by multiplying by in the multiplicative inverse of det($A$) in GF(7). Now, use the method above to find the inverse of $A$ in GF(7). You may use MATLAB or any other software. Describe your method.

In [71]:
def inverse_rec(matrix,p) :
    i=0
    output = np.zeros([len(matrix),len(matrix)],dtype=int)
    det = recursive_det(matrix,p)
    inv_det = find_modulo_inverse(det, p)
    while i<len(matrix):
        j=0
        while j<len(matrix) :
            submatrix = np.delete((np.delete(matrix, i,axis=0)),j,axis=1)
            #print(output)
            #print(submatrix)
            val = recursive_det(submatrix,p)*((-1)**(i+j))
            #print(f'val={val}')
            output[i][j] = inv_det*val % p
            j+=1
        i+=1
    output = np.array(output)
    return output.transpose()

In [72]:
A = np.array([[3,1,6],[3,0,4],[1,3,2]])
A_inv = inverse_rec(A,7)
print('A^-1 = ')
print(A_inv)

print('')
#Testing to see if inverse produced identity matrix.
print('A*A^-1 = ')
print(np.matmul(A,A_inv) % 7)

print(recursive_det(A,7))

A^-1 = 
[[1 1 2]
 [6 0 3]
 [1 3 2]]

A*A^-1 = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]
2


> $ A^{-1} = \begin{bmatrix}
1 & 1 & 2 \\
6 & 0 & 3 \\
1 & 3 & 2 
\end{bmatrix}  $

> $A \cdot A^{-1} =
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 
\end{bmatrix}$

Using a RREF calculator, https://www.emathhelp.net/en/calculators/linear-algebra/reduced-row-echelon-form-rref-calculator/?i=%5B%5B3%2C2%2C5%5D%2C%5B3%2C5%2C6%5D%2C%5B2%2C3%2C1%5D%5D&reduced=on ,

we get:

$A \cdot A^{-1} = \begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 
\end{bmatrix}$


## Problem 2

Consider a general block cipher algorithm that maps $n$ bits to $n$ bits.

(a) How many different such mapping (including the identity map) exist?

> $2^{N}!$

(b) Let $K(n)$ be your answer to part (a). Bob and Alice want to communicate privately. In order to set the mapping, they shared a long table including all possible $K(n)$ mappings and associated a number between $0$ and $K(n) − 1$ to each. Then, they wrote the corresponding number for each mapping in Binary representation. For instance, if $K(n) = 1000$, mapping number $347$ is represented by $’0101011011’$. Then, each time Bob wants to send a private message to Alice, he chooses one of these mappings and sends her its binary representation through a secure channel. How many bits does Bob have to send for each mapping?

> ceiling($log_{2}(2^{N}!-1)$). Because we have $2^{N}!$ mappings, and number x can be represented by ceiling($log_2(x)$) bits


(c) Recall that we mentioned in class that $n$-to-$n$ bits mappings can be also represented by $n$ × $2n$ bits (similar to Table. 4.1 of the textbook). Is this number less or greater than what you found in part (b)? Explain the discrepancy.

> This number would be greater, because for each possible mapping, there are twice as many choices to choose from.

## Problem 3

Consider the GF($2^{8}$) generated by $q(x) = x^{8} + x^{7} + x^{2} + x + 1$. Evaluate the following quantities:

(a) $\{BC\} + \{2E\}$.

In [73]:
def hex_add(A,B,p) : #Adds A to B in GF(p), where A and B are hexadecimal values.
    A_array = [x for x in bin(A)]
    B_array = [x for x in bin(B)]
    A_array.pop(0) #get rid of '0b' from the arrays
    A_array.pop(0)
    B_array.pop(0)
    B_array.pop(0)
    A_array = [int(x) for x in A_array] #convert each element to integer
    B_array = [int(x) for x in B_array]


    while len(A_array) < 8 : #add zeros to make array an 8 bit number
        A_array.insert(0,0)
    while len(B_array) < 8 :
        B_array.insert(0,0)

    output = [0] * 8
    i=0
    while i<8 :
        output[i] = (A_array[i] + B_array[i])%p #add each bit, modulus 2
        i+=1
    return hex(int('0b' + ''.join(str(x) for x in output), 2))

print(hex_add(0xBC,0x2E,2))


0x92


> $\{BC\} + \{2E\} = \{92\}$

(b) $\{0A\} \cdot \{A3\}$.

In [74]:
#These are mostly altered versions of my code from HW2.

def find_degree(polynomial) : #Finds the degree of a polynomial vector by ignoring finding the first non-zero value.
    shift = 0
    for i in polynomial :
        if i!=0 :
            return len(polynomial)-shift-1
        else : 
            shift +=1
    return 0

def remove_zeros(polynomial) : #Removes all zeros in a polynomial vector preceeding the first non-zero element.
    #e.g. [0, 0, 0, 2, 0, 3, 0] --> [2, 0, 3, 0]

    shift = 0
    for i in polynomial :
        if i!=0 :
            while shift>0 :
                polynomial.pop(0)
                shift-=1
            break
        else : 
            shift +=1
    return polynomial

def MyGFAdd(A, B, M, p) : # Add A and B modulo p.
    #==INPUTS==
    #prime: prime number from p in GF(p)
    #A: an array of polynomial coefficients e.g. 2x^2 + 1 --> [2 0 1]
    #B: same formate as A
    #M: do not care
    #p: p in GF(p), prime number
    #==OUTPUT==
    #Returns polynomial coefficient array that is 


    degree_A = find_degree(A) #highest degree of A
    degree_B = find_degree(B) #highest degree of B
    degree_diff = abs(degree_A - degree_B)
    A = remove_zeros(A) #remove preceeding zeros
    B = remove_zeros(B)

    #Add A and B differently depending on which one is longer or shorter.
    if degree_A > degree_B : 
        #print('A > B')
        output = [0] * len(A) # A is a longer array
        i=0
        while i<len(A) :
            if i < degree_diff : output[i] = A[i] #don't start adding B until A and B are at the same degree.
            else : output[i] = A[i]+B[i-degree_diff]
            i+=1

    else :
        if degree_A < degree_B : 
            #print('A < B')
            output = [0] * len(B) # B is a logner array
            i=0
            while i<len(B) :
                if i < degree_diff : output[i] = B[i] #don't start adding B until A and B are at the same degree.
                else : output[i] = B[i]+A[i-degree_diff]
                i+=1
        else : #degree_A = degree_B
            #print('A=B')
            output = [0] * len(A)
            i=0
            for element in A :
                output[i] = element+B[i]
                i+=1
    
    # apply modulus to output.
    i=0
    while i<len(output) :
        output[i] = np.mod(output[i], p)
        i+=1

    return output

def single_mult(k,degree_k,A) : #Multiplies one value into a full polynomial
    #==hexS==
    #k: scalar coefficent of single value.
    #degree_k: degree of 
    #A: polynomial coefficient vector.
    #==OUTPUT==
    #Returns a polynomial coefficient array: A*kx^(degree)
    degree_A = find_degree(A)
    A = remove_zeros(A)
    output = [0] * (degree_A+degree_k+1)

    if k == 0 :
        return output

    i=0
    while i<len(A):
        output[i] = k*A[i]
        i+=1

    return output

def raw_poly_mult_GF(A, B, GF) :
    #Multiplies two polynomials, ignoring degree > M
    degree_A = find_degree(A)
    degree_B = find_degree(B)
    
    if degree_A > degree_B : 
        #print('A > B')
        partial_output = [[0 for _ in range(degree_A+degree_B+1)] for _ in range(len(B))]
        i=0
        for element in B :
                partial_output[i] = single_mult(element,(degree_B-i),A)
                i+=1

    else :
        if degree_A < degree_B : 
            #print('A < B')
            partial_output = [[0 for _ in range(degree_A+degree_B+1)] for _ in range(len(A))]
            i=0
            for element in A :
                partial_output[i] = single_mult(element,(degree_A-i),B)
                i+=1

        else : #degree_A = degree_B
            #print('A=B')
            partial_output = [[0 for _ in range(degree_A+degree_B+1)] for _ in range(len(A))]
            i=0
            for element in A :
                partial_output[i] = single_mult(element,(degree_A-i),B)
                i+=1


    max_degree = 0
    for element in partial_output :
        deg = find_degree(element)
        if deg > max_degree :
            max_degree = deg
    #print(f'max_deg={max_degree}')

    output = [0] * (max_degree+1)

    output = partial_output[0]
    #print(f'partial_output={partial_output}')
    i=1
    while i<(len(partial_output)) :
        output = MyGFAdd(output,partial_output[i],0,GF)
        #print(f'output={output}')
        i+=1


    return output

def find_highest_coefficient(M,p) :
    #Finds what the highest degree coefficient is equal to. Assumes that that coefficient = 1.
    #e.g. 
    # hex: [1, 1, 1] output: [2, 2]
    #      (x^2 + x + 1)   =   (2x+2)

    M[0] = 0
    M = list(np.multiply(M,(-1)))
    M = np.mod(remove_zeros(M),p)
    return M

def MyGFMult(A, B, M, p) :
    #Returns the multiplication of A and B in GF(p^k)
    #==INPUTS==
    #A: coefficient polynomial array A
    #B: coefficient polynomial array B
    #M: m(x) is an irreducible polynomial in GF(p^k)
    #p: prime number p in GF(p)
    #==OUTPUTS==
    #Returns a polynomial coefficient array of the form Poly [c_n, c_n-1, ..., c1 , c0] --> Poly = c_n*x^n + c_(n-1)*x^(n-1) + ... c1*x + c0


    #find the degree of each polynomial
    A_array = [x for x in bin(A)]
    B_array = [x for x in bin(B)]
    A_array.pop(0) #get rid of '0b' from the arrays
    A_array.pop(0)
    B_array.pop(0)
    B_array.pop(0)
    A_array = [int(x) for x in A_array] #convert each element to integer
    B_array = [int(x) for x in B_array]

    degree_A = find_degree(A_array)
    degree_B = find_degree(B_array)
    degree_M = find_degree(M)
    degree_diff = abs(degree_A - degree_B)

    #get rid of any preceeding zeros
    A = remove_zeros(A_array)
    B = remove_zeros(B_array)
    M = remove_zeros(M)

    raw_mult = raw_poly_mult_GF(A,B,p)
    raw_degree = find_degree(raw_mult)
    inv_M = find_highest_coefficient(M,p) # not actually an inverse, but rather what the highest degree of M equals. e.g. M == x^2 + x + 1, inv_M = 2x+2
    
    #if there needs to be substitutions.
    #e.g. for GF(3), M == x^2 + x + 1, (x^2 = 2x+2), and raw_mult == x^3 --> raw_mult = x(x^2) = x(2x+2) = 2x^2 + 2x = 4x + 4 + 2x = 6x + 4 = 1 (mod 3)
    if raw_degree >= degree_M : 
        degree_diff = raw_degree - degree_diff
        i=0
        while degree_diff>=0 :
            single_elem = [0] *(len(raw_mult)-len(M)+1)
            single_elem[0] = raw_mult[0]

            raw_mult[0] = 0 #remove highest degree of raw_mult for substitution

            #(single_elem*inv_M) is the replacement for the highest degree power of raw_mult, whose result
            #is added back to raw_mult to fully apply the subsitution
            raw_mult = MyGFAdd(raw_poly_mult_GF(inv_M,single_elem,p),raw_mult,M,p) #
            raw_mult = remove_zeros(raw_mult) #important for reducing len(raw_mult) to proper size.
            
            raw_degree = find_degree(raw_mult)
            degree_diff = raw_degree - degree_M
            
            i+=1

    return raw_mult

In [75]:
Mx = [1,1,0,0,0,0,1,1,1]
p = 2
print('Answer = ')
print(hex(int('0b'+''.join([str(x) for x in MyGFMult(0x0A,0xA3,Mx, p)]),2)))

Answer = 
0xcb


> $\{0A\} \cdot \{A3\} = \{CB\}$

In [76]:
Mx2 = [1,0,0,0,1,1,0,1,1]
p = 2
MyGFMult(0x03,0x6E,Mx2, p)

[1, 0, 1, 1, 0, 0, 1, 0]

(c) $\{84\}^{−1} = \{73\}$. See problem 4d for calculation.

## Problem 4

Consider GF($3^{3}$) generated by $m(x) = x^{3} + 2x^{2} + 1$, and let $\alpha$ be a root of $m(x)$. Let $a = 2\alpha^{2} + \alpha + 1$. We have already seen that we can find the multiplicative inverse of $a$ as follows: Let $b = b_{2}\alpha^{2} + b_{1}\alpha + b_{0}$ is $a^{−1}$. Then, we there should exists some $q$ such that

$a \cdot b = m(α) \cdot q + 1.$

(a) We can argue that $q$ should be of the form of $q = q_{1}\alpha + q_{0}$. Why $q$ does not have any term of $\alpha^{2}$?

>Because then $m(\alpha)\cdot q$ would be of the form: $c_{4} \alpha ^{4} + c_{3}\alpha ^{3} + c_{2}\alpha ^{2} c_{1} \alpha + c_{0}$ which is a higher degree than $a \cdot b$

Now, to solve for (1), we can write

$a \cdot b = (2\alpha^{2} + \alpha + 1) \cdot (b_{2}\alpha^{2} + b_{1}\alpha + b_{0}) = (2b_{2})\alpha^{4} + (2b_{1} + b_{2})\alpha^{3} + (2b_{0} + b_{1} + b_{0})\alpha^{2} + (b_{1} + b_{0})\alpha + b_{0}$

$m(α) \cdot q + 1 = q_{1}\alpha^{4} + (2q_{1} + q_{0})\alpha^{3} + (2q_{0})\alpha^{2} + q_{1}\alpha + (q_{0}) + 1.$

So, we need to determine $(b_{2}, b_{1}, b_{0})$ and $(q_{1}, q_{0})$ such that

$2b_{2} = q_{1}$

$2b_{1} + b_{2} = 2q_{1} + q_{0}$

$2b_{0} + b_{1} + b_{0} = 2q_{0}$

$b_{1} + b_{0} = q_{1}$

$b_{0} = q_{0} + 1.$

(b) Write the equations in (2) as a linear system, i.e, $A \cdot x = b$, where $x = [b_{0}, b_{1}, b_{2}, q_{0}, q_{1}]^{T}$ . Find $A$ and $b$.

> $ A = \begin{bmatrix}
0 & 0 & 2 & 0 & 2 \\
0 & 2 & 1 & 2 & 1 \\
2 & 1 & 1 & 1 & 0 \\
1 & 1 & 0 & 0 & 2 \\
1 & 0 & 0 & 2 & 0 \\
\end{bmatrix}
;   b = \begin{bmatrix}
0 \\
0 \\
0 \\
0 \\
1
\end{bmatrix}
$

(c) Use Problem 1 to find the inverse of $A$ and solve the system for $x$. What is $b$?

In [77]:
A = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[16,15,14,13]]
print(recursive_det(A,3))
print(np.linalg.det(A)%3)

0
0.0


In [78]:
A = [[0,0,2,0,2],[0,2,1,2,1],[2,1,1,1,0],[1,1,0,0,2],[1,0,0,2,0]] #"A" from 4b)
print(recursive_det(A,3))
print(round(np.linalg.det(A)%3))
b = [[0],[0],[0],[0],[1]] #"b" from 4b)
inv_a = inverse_rec(A,3)
print('A = ')
print(np.array(A))
print('')
print('A^-1 = ')
print(inv_a)
print('')
print('A*A^-1 = ')
print(np.matmul(inv_a,A)%3)
print('')
print('x = ')
print(np.matmul(inv_a,b))

2
2
A = 
[[0 0 2 0 2]
 [0 2 1 2 1]
 [2 1 1 1 0]
 [1 1 0 0 2]
 [1 0 0 2 0]]

A^-1 = 
[[2 0 2 1 2]
 [0 2 1 2 2]
 [0 1 0 1 2]
 [2 0 2 1 1]
 [2 2 0 2 1]]

A*A^-1 = 
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]

x = 
[[2]
 [2]
 [2]
 [1]
 [1]]


> $ A^{-1} = \begin{bmatrix}
2 & 0 & 2 & 1 & 2 \\
0 & 2 & 1 & 2 & 2 \\
0 & 1 & 0 & 1 & 2 \\
2 & 0 & 2 & 1 & 1 \\
2 & 2 & 0 & 2 & 1 \\
\end{bmatrix} $

> $ A \cdot A^{-1} = 
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}$

> $ x = 
\begin{bmatrix}
2 \\
2 \\
2 \\
1 \\
1 \\
\end{bmatrix}$

$b_{0} = 2, b_{1} = 2, b_{2} = 2$

$b = 2\alpha^{2} + 2\alpha + 2$

(d) Write a computer program that computes the multiplicative inverse in a general finite field. Consider GF($p^{k}$) generated by $m(x) = m_{k}x^{k} + · · · + m_{1}x + m_{0}$ which is represented as $M = [m_{k}, m_{k−1}, . . . , m_{1}, m_{0}]$. For a value $a = a_{k−1}\alpha^{k−1} + · · · + a_{1}\alpha + a_{0}$, which is represented as $A = [a_{k−1}, . . . , a_{1}, a_{0}]$, compute $b = b_{k−1}\alpha^{k−1} + · · · + b_{1}\alpha + b_{0}$ such that $a \cdot b = 1$ and return $B = [b_{k−1}, . . . , b_{1}, b_{0}]$.

In [79]:
def poly_str(polynomial) :
    #Returns a string statement to be printed. Provides a more readable version than just printing a polynomial.
    i=0
    output = ''
    while i<len(polynomial) :
        j=0
        while j<len(polynomial[i]) :
            if polynomial[i][j] != [0] :
                if j==0 :
                    output = output + '('
                output = output + str(polynomial[i][j][0])+'*'+str(polynomial[i][j][1])
                if j!=(len(polynomial[i])-1) :
                    output = output + ' + '
            j+=1
        output = output + ')'
        if i!=len(polynomial)-1 :
            output = output + '*α^' + str(len(polynomial)-i-1)
            output = output + ' + ' 
        i+=1

    return output


def mult_inverse_GF(a, M, p) :
    #Returns the multiplicative inverse of a in a general finite field GF(p^k) generated by m(x) = mkx^k + ... m1x + m0
    #INPUTS:
    #a[]: polynomial coefficients in GF(p) to find inverse of
    #M[]: polynomial coefficients of m(x)
    #p: p in GF(p^k), integer.

    degree_a = find_degree(a)
    degree_M = find_degree(M)
    mult = [[] for x in range(0,2*len(a)-1)] # matrix result of a*b.

    b = [['b' + str(k-1), k-1] for k in range(degree_M,0,-1)] # [['bk-1',k-1],['bk-1',k-2],...,['b1',1],['b0',0]]
    
    if degree_a<degree_M-1 : 
        #add zeros to make "a" of degree k-1, where k is the degree of M.
        diff=degree_M-degree_a
        i=0
        while i<diff : 
            #add zeros to make "a" of degree k-1, where k is the degree of M.
            a.insert(0,0)
            i+=1

    i=0
    while i<len(a) : #Multiply coefficents of a by coefficients of b, but don't add up like values yet.
        #Store them in 2d array "mult", of the form: mult[<descending polynomial dexponent>][<numeric coefficient>,<any of {b0, b1, ..., bk-1}>]
        # e.g. if mult = [[0],[[2,'b1'],[1,'b2']],[1,'b0']] ---> 0*alpha^2 + (2b1 + 1b2)*alpha^1 + 1b0
        j=0
        while j<len(b) :
            exp_a = len(a)-i #exponent value of current element of a
            exp_b = b[j][1] #exponent value of current element of b
            exp_total = exp_a + exp_b
            if a[i] != 0 : # do not multiply if coefficient of a=0
                mult[len(mult)-exp_total].append([a[i],b[j][0]])
            #print(mult)
            j+=1
        i+=1


    i=0
    while i<len(mult) : #replace unused elements with zero
        if mult[i] == [] : mult[i] = [[0]]
        i+=1

    i=0
    while i<len(mult) : #remove preceeding zeros.
        #e.g. if mult = [[[0]],[[1,'b0],[2,'b1']], [[0]]]
        #   then mult = [[[1,'b0],[2,'b1']], [[0]]]
        if mult[i] != [[0]] :
            break
        mult.pop(0)
        i+=1

    #Replace all elements > degree M with elements in lower degree.
    inv_M = find_highest_coefficient(M,p) # not actually an inverse, but rather what the highest degree of M equals. e.g. M == x^2 + x + 1, inv_M = 2x+2
    degree_mult = find_degree(mult)
    if degree_mult >= degree_M : 
        degree_diff = degree_mult - degree_M
        while degree_diff>=0 :
            mult = remove_zeros(mult)
            single_elem = [0]
            single_elem[0] = mult[0]
            mult[0] = 0 #remove highest degree of raw_mult for substitution
            mult = remove_zeros(mult)
            if single_elem[0][0] != [0] :
                #(single_elem*inv_M) is the replacement for the highest degree power of raw_mult, whose result
                #is added back to raw_mult to fully apply the subsitution
                i=0
                while i<len(inv_M) :
                    j=0
                    while j<len(single_elem[0]) :
                        exp_M = len(inv_M)-i-1 #exponent value of current element of a
                        exp_single_elem = len(mult)-degree_M #exponent value of current element of b
                        exp_total = exp_M + exp_single_elem
                        index = len(mult) - exp_total - 1

                        if inv_M[i] != 0: # do not multiply if coefficient of a=0
                            mult[index].append([inv_M[i]*single_elem[0][j][0],single_elem[0][j][1]])
                        j+=1
                    i+=1
            degree_mult = find_degree(mult)
            degree_diff = degree_mult - degree_M # loop condition

    i=0
    while i<len(mult) :
        j=0
        while j<len(mult[i]) :
            k=j
            if mult[i][j] != [0] :
                total = mult[i][j][0] # running total for how many times that item occurs
                coefficient = mult[i][j][1] #e.g. "b0" or "b1"
                while k<len(mult[i]) :
                    if mult[i][k] != [0] :
                        if (coefficient == mult[i][k][1]) & (k!=j) : 
                            total = total + mult[i][k][0]
                            mult[i][k] = [0]
                    k+=1
                mult[i][j][0] = total
            j+=1
        i+=1

    #remove empty values
    i=0
    while i<len(mult) :
        j=0
        while j<len(mult[i]) :
            if mult[i][j] == [0] :
                mult[i].pop(j)
                j-=1
            j+=1
        i+=1


    solution_matrix = [[0 for x in range(0,len(mult)+1)] for y in range(0,len(mult))]
    i=0
    while i<len(mult) :
        j=0
        while j<len(mult[i]) :
            coefficient_index = int(mult[i][j][1][1:None])
            solution_matrix[i][coefficient_index] = np.mod((mult[i][j][0]),p)
            j+=1
        if i!=(len(mult)-1) :
            solution_matrix[i][len(solution_matrix)] = 0
        else : solution_matrix[i][len(solution_matrix)] = 1
        i+=1

    #Create a copy of solution_matrix, called solution_matrix_2 which uses the Matrix Inverse Method
    solution_matrix_2 = [[0 for x in range(0,len(mult))] for y in range(0,len(mult))]
    i=0
    while i<len(solution_matrix) :
        j=0
        while j<(len(solution_matrix)) :
            solution_matrix_2[i][j] = solution_matrix[i][j]
            j+=1
        i+=1

    #Calculate the RREF of solution_matrix to get the answer
    solution_matrix = sp.Matrix(solution_matrix)
    RREF = solution_matrix.rref()
    output = [0 for x in range(0,len(RREF[0][0:None,0]))]
    i=0
    while i<(len(RREF[0][0:None,0])) :
        j=0
        while j<(len(RREF[0][i,0:None])-1) :
            output[len(output)-i-1] = np.mod(RREF[0][i,(len(RREF[0][i,0:None])-1)],p)
            j+=1

        i+=1

    #Calculate answer using Inverse Matrix
    inv_matrix = inverse_rec(solution_matrix_2,p)
    one_matrix = [[0] for x in range(0,len(solution_matrix_2))]
    one_matrix[len(one_matrix)-1] = [1] # matrix of form: [[0], [0], [0], ... , [0], [1]] (one_matrix = 1 in a system of equations.)
    output2 = np.matmul(np.array(inv_matrix),np.array(one_matrix))
    output2 = np.flip(output2.transpose())


    return output, output2

#prints out 8 bit array into hexadecimal based on the answer format from mult_inverse_GF()
def print_mult_inv_ans(input, answer) :
    input_int = int(str(hex(int('0b'+''.join([str(x) for x in input]),2))),0)
    # method1 = int(str(hex(int('0b'+''.join([str(x) for x in answer[0]]),2))),0)
    method2 = int(str(hex(int('0b'+''.join([str(x) for x in answer[1][0]]),2))),0)
    lb = '{'
    rb = '}'
    print(f'Finding {lb}{hex(input_int)}{rb}^(-1),')
    # print(f'using RREF, answer = {lb}{hex(method1)}{rb}')
    print(f'using Matrix Inverse, answer = {lb}{hex(method2)}{rb}')
    return

In [80]:
a =   [1,0,1,1] #{0xb}
M = [1,0,0,1,1]
answer = mult_inverse_GF(a,M,2)
print_mult_inv_ans(a,answer)

Finding {0xb}^(-1),
using Matrix Inverse, answer = {0x5}


In [81]:
b =   [0,1,0,1] #{0x5}
M = [1,0,0,1,1]
answer2 = mult_inverse_GF(b,M,2)
print_mult_inv_ans(b,answer2)

Finding {0x5}^(-1),
using Matrix Inverse, answer = {0xb}


### Calculating 3c

In [82]:

a = [1,0,0,0,0,1,0,0] # = {84}
Mx = [1,1,0,0,0,0,1,1,1] # m(x) = x^8 + x^7 + x^2 + x + 1
answer = mult_inverse_GF(a,Mx,2)
print_mult_inv_ans(a,answer) #print it out all nice


Finding {0x84}^(-1),
using Matrix Inverse, answer = {0x73}


In [83]:
input = [0,1,1,1,0,0,1,1] # = {73}
Mx = [1,1,0,0,0,0,1,1,1] # m(x) = x^8 + x^7 + x^2 + x + 1
answer2 = mult_inverse_GF(input,Mx,2)
print_mult_inv_ans(input,answer2) #print it out all nice

Finding {0x73}^(-1),
using Matrix Inverse, answer = {0x84}


## Problem 5

Recall the S-box used in the Advanced Encryption Standard (AES). It maps a pair $\{xy\}$ to $\{x^{′}y^{′}\}$ = $S(\{xy\})$ according to the following steps:

• You first find $\{uv\} = \{xy\}^{−1}$.

• Write $\{uv\}$ in binary $\{uv\} = [b_{7}, . . . , b_{0}]$.

• Compute

$ \begin{bmatrix}
b_{0}^{'} \\
b_{1}^{'} \\
b_{2}^{'} \\
b_{3}^{'} \\
b_{4}^{'} \\
b_{0}^{'} \\
b_{0}^{'} \\
b_{0}^{'} \\
\end{bmatrix} = \begin{bmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 1 & 1 \\
1 & 1 & 0 & 0 & 0 & 1 & 1 & 1 \\
1 & 1 & 1 & 0 & 0 & 0 & 1 & 1 \\
1 & 1 & 1 & 1 & 0 & 0 & 0 & 1 \\
1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 \\
0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 
\end{bmatrix} \cdot
\begin{bmatrix}
b_{0} \\
b_{1} \\
b_{2} \\
b_{3} \\
b_{4} \\
b_{5} \\
b_{6} \\
b_{7} 
\end{bmatrix} +
\begin{bmatrix}
1 \\
1 \\
0 \\
0 \\
0 \\
1 \\
1 \\
0 
\end{bmatrix}$ (mod 2)

• Compute $\{x^{′}y^{′}\} = [b^{′}_{7}, . . . , b^{′}_{0}]$.

Write a computer program to generate the S-box. It should take $\{xy\}$ and return $\{x^{′}y^{′}\}$.

In [196]:
def hex_to_bin_list(hex, length) :
    #Converts hexadecimal value(or a decimal) 
    hex = list(bin(hex))
    hex.pop(0)
    hex.pop(0)
    if len(hex)<length:
        diff = (length-len(hex))
        for i in range(0,diff) :
            hex.insert(0,'0')

    for i in range(0,len(hex)) :
        hex[i] = int(hex[i])
    return hex

def bin_list_to_hex(bin) :
    #converts a binary list to a hexidecimal number.
    return hex(int('0b'+''.join([str(x) for x in bin]),2))

def S(input) :
    M = [1,0,0,0,1,1,0,1,1]
    l = np.array([[1],[1],[0],[0],[0],[1],[1],[0]]) #63
    S_array = np.array([[1,0,0,0,1,1,1,1],[1,1,0,0,0,1,1,1],[1,1,1,0,0,0,1,1],[1,1,1,1,0,0,0,1],[1,1,1,1,1,0,0,0],[0,1,1,1,1,1,0,0],[0,0,1,1,1,1,1,0],[0,0,0,1,1,1,1,1]])

    input = hex_to_bin_list(input, len(M))
    inverse = mult_inverse_GF(input,M,2)
    inverse = np.vstack(np.flip(np.array(inverse[1][0])))
    output = np.matmul(S_array,inverse) % 2
    output = (output + l) % 2
    output = np.flip(np.reshape(output,8))
    return bin_list_to_hex(output)

In [262]:
# Testing S(input) to see if it works.

inp = 0x95 #{95}
outp = S(inp) #expected answer: {2A}
lb = '{'
rb = '}'
print(f'Input = {lb}{hex(inp)}{rb}')
print(f'Output = {lb}{outp}{rb}')

Input = {0x95}
Output = {0x2a}


In [254]:
def generate_SBox() :
    matrix = [['00' for x in range(0,16)] for y in range(0,16)]
    for i in range(0,16) :
        for j in range(0,16) :
            value = np.concatenate((np.array(hex_to_bin_list(i, 4)),np.array(hex_to_bin_list(j, 4))),axis=0)
            value = int(bin_list_to_hex(value),16)
            matrix[i][j] = S((value))
    return np.array(matrix)
            

In [255]:
#This takes 13.5 minutes to compute!!
SBox_array = generate_SBox()

In [261]:
def print_SBox(SBox) : # Prints an S box out.
    topline = '   |'
    for num in range(0,16) :
            topline = topline + ' ' + hex(num)[2] + '   '
    print(topline)
    divider = '---|'
    for y in range(0,79) :
        divider = divider + '-'
    print(divider)
    for i in range(0,len(SBox)) :
        string = ' ' + hex(i)[2] + ' |'
        for j in range(0,len(SBox[i])) :
            value = SBox[i][j]
            if len(value)<4 : # adds in necessary zeros e.g. 0x7 --> 0x07
                 value = value[:2] + '0' + value[2:None]
            string = string + value + ' '
        print(string)
    return

print_SBox(SBox_array)

   | 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f   
---|-------------------------------------------------------------------------------
 0 |0x63 0x7c 0x77 0x7b 0xf2 0x6b 0x6f 0xc5 0x30 0x01 0x67 0x2b 0xfe 0xd7 0xab 0x76 
 1 |0xca 0x82 0xc9 0x7d 0xfa 0x59 0x47 0xf0 0xad 0xd4 0xa2 0xaf 0x9c 0xa4 0x72 0xc0 
 2 |0xb7 0xfd 0x93 0x26 0x36 0x3f 0xf7 0xcc 0x34 0xa5 0xe5 0xf1 0x71 0xd8 0x31 0x15 
 3 |0x04 0xc7 0x23 0xc3 0x18 0x96 0x05 0x9a 0x07 0x12 0x80 0xe2 0xeb 0x27 0xb2 0x75 
 4 |0x09 0x83 0x2c 0x1a 0x1b 0x6e 0x5a 0xa0 0x52 0x3b 0xd6 0xb3 0x29 0xe3 0x2f 0x84 
 5 |0x53 0xd1 0x00 0xed 0x20 0xfc 0xb1 0x5b 0x6a 0xcb 0xbe 0x39 0x4a 0x4c 0x58 0xcf 
 6 |0xd0 0xef 0xaa 0xfb 0x43 0x4d 0x33 0x85 0x45 0xf9 0x02 0x7f 0x50 0x3c 0x9f 0xa8 
 7 |0x51 0xa3 0x40 0x8f 0x92 0x9d 0x38 0xf5 0xbc 0xb6 0xda 0x21 0x10 0xff 0xf3 0xd2 
 8 |0xcd 0x0c 0x13 0xec 0x5f 0x97 0x44 0x17 0xc4 0xa7 0x7e 0x3d 0x64 0x5d 0x19 0x73 
 9 |0x60 0x81 0x4f 0xdc 0x22 0x2a 0x90 0x88 0x46 0xee 0xb8 0x14 0x